In [1]:
import socket
import sys
from collections import namedtuple
from struct import unpack
import asyncore
import rospy
from std_msgs.msg import Float32
from std_msgs.msg import String
from std_srvs.srv import Trigger

In [6]:
Message = namedtuple('Message', ['position1','speed1','torque1','position2','speed2','torque2','first_letter','second_letter'])


class HTTPClient(asyncore.dispatcher):

    def __init__(self, host="192.168.1.120", port=8000, message_size=50):
        #self.pub0 = rospy.Publisher('/rail1/position', Float32, queue_size=10)
        #self.pub1 = rospy.Publisher('/rail1/speed', Float32, queue_size=10)
        #self.pub2 = rospy.Publisher('/rail1/torque', Float32, queue_size=10)
        #self.pub3 = rospy.Publisher('/rail2/position', Float32, queue_size=10)
        #self.pub4 = rospy.Publisher('/rail2/speed', Float32, queue_size=10)
        #self.pub5 = rospy.Publisher('/rail2/torque', Float32, queue_size=10)
        asyncore.dispatcher.__init__(self)
        self.create_socket()
        self.message_size=message_size #message_size=50
        self.connect( (host, port) ) #connect to port 8000, host 192.168.1.120 
        #self.buffer = bytes('testing','utf8')
        self.counter=0
        rospy.init_node('start_stop_server')
        self.s_start = rospy.Service('start_stop', Trigger, self.start_stop_communication)
        rospy.init_node('mode_server')
        self.s_mode = rospy.Service('mode', Trigger, self.change_mode)
        self.last_message=""
    
    def start_stop_communication(self):
        'this methond allows or stops communication between ROS and the PLC'
        print("Triggering Start/Stop")
        self.shutdown(self.SHUT_RDWR)
        self.close()
        return Start_StopResponse(True, "Trigger successful")
        
    def change_mode(self):
        print("Changing Mode")
        return ModeResponse(True, "Mode changed successfully")
        
    def handle_connect(self):
        print("connected")
        pass
        
    def decode_message(self,message):
        'this method decodes the message received through TCP/IP'
        if(len(message)==self.message_size):
            touple_from_tcp=unpack('ddddddcc', message)
            my_message=Message(*touple_from_tcp)
            return my_message
        return False
    
    def handle_read(self):
        'this method receives a message through TCP/IP and sends it to ROS'
        byte_message=self.recv(self.message_size)
        self.last_message=self.decode_message(byte_message)
        if(self.last_message):
            try:
                pass
                #print(self.last_message.position1)
                #self.pub0.publish(self.last_message.position1)
                #self.pub1.publish(self.last_message.speed1)
                #self.pub2.publish(self.last_message.torque1)
                #self.pub3.publish(self.last_message.position2)
                #self.pub4.publish(self.last_message.speed2)
                #self.pub5.publish(self.last_message.torque2)
            except:
                print(":-(")
        self.counter+=1
        
    #def writable(self):
        #return (len(self.buffer) > 0)

    #def handle_write(self):
        #sent = self.send(self.buffer)
        #self.buffer = self.buffer[sent:]


In [8]:
client = HTTPClient('192.168.1.120',8000)
asyncore.loop()

ServiceException: service [/start_stop] already registered

In [2]:
#received_message=sock.recv(50)
#print(received_message)

In [3]:
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as sock:
        ip="192.168.1.120"
        port=8000
    # Connect the socket to the port where the server is listening
        server_address = (ip,port)
        print( 'connecting to %s port %s' % server_address)
        somesocket=sock.connect(server_address)
        #sock.sendall("testing".encode())
        received_message=sock.recv(50)



connecting to 192.168.1.120 port 8000


In [4]:
print(len(received_message))

50


In [5]:
touple_from_tcp=unpack('ddddddcc', received_message)
touple_from_tcp

(140.29997355707184, 0.0, -31.1, 150.69999303499608, 0.0, -4.5, b'O', b'K')

In [6]:
Message = namedtuple('Message', ['position1','speed1','torque1','position2','speed2','torque2','first_letter','second_letter'])
my_message=Message(*touple_from_tcp)
list(my_message)

[140.29997355707184, 0.0, -31.1, 150.69999303499608, 0.0, -4.5, b'O', b'K']

In [7]:
my_message.position1

140.29997355707184

In [ ]:
client.socket.close()

In [ ]:
client.counter